In [2]:
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
  Using cached wordcloud-1.8.2.2-cp39-cp39-win_amd64.whl (153 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install neattext
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from keras.utils import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('short.csv')
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,class
0,1,1,3,Am I weird I don't get affected by compliments...,non-suicide
1,2,2,4,Finally 2020 is almost over... So I can never ...,non-suicide
2,3,3,8,i need helpjust help me im crying so hard,suicide
3,4,4,9,Oh Fuck! can't take this anymore.I am tired of...,suicide
4,5,5,11,I am gonna hurt myself,suicide


In [4]:
data=data.drop(["Unnamed: 0.1","Unnamed: 0"], axis=1)
data.head()

,Unnamed: 0.2,text,class
0,1,Am I weird I don't get affected by compliments...,non-suicide
1,2,Finally 2020 is almost over... So I can never ...,non-suicide
2,3,i need helpjust help me im crying so hard,suicide
3,4,Oh Fuck! can't take this anymore.I am tired of...,suicide
4,5,I am gonna hurt myself,suicide


In [5]:
data['class'].value_counts()

non-suicide    55200
suicide        26442
Name: class, dtype: int64

In [6]:
data['class'].value_counts().index.values

array(['non-suicide', 'suicide'], dtype=object)

In [7]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [8]:
train_data['class'].value_counts().index.values

array(['non-suicide', 'suicide'], dtype=object)

In [9]:

def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
#         sent=nfx.remove_shortwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [10]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████| 16329/16329 [00:00<00:00, 52006.72it/s]


In [11]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [12]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

In [13]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [14]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['class'])
test_output=lbl_target.transform(test_data['class'])

In [15]:
# glove_embedding={}
with open('glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [16]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [17]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [18]:
suicide_model=Sequential()
suicide_model.add(Input(shape=(50,)))
suicide_model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
suicide_model.add(LSTM(20,return_sequences=True))
suicide_model.add(GlobalMaxPooling1D())
#     model.add(Dropout(0.3))
suicide_model.add(Dense(256,activation='relu'))
#     model.add(Dropout(0.2))
suicide_model.add(Dense(1,activation='sigmoid'))
suicide_model.compile(optimizer=keras.optimizers.SGD(0.05,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
r=suicide_model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=64,callbacks=[early_stop,reducelr])

Epoch 1/20
1021/1021 [==============================] - 35s 30ms/step - loss: 0.3430 - accuracy: 0.8476 - val_loss: 0.2400 - val_accuracy: 0.9032 - lr: 0.0500
Epoch 2/20
1021/1021 [==============================] - 34s 34ms/step - loss: 0.2086 - accuracy: 0.9153 - val_loss: 0.2044 - val_accuracy: 0.9157 - lr: 0.0500
Epoch 3/20
1021/1021 [==============================] - 35s 34ms/step - loss: 0.1846 - accuracy: 0.9255 - val_loss: 0.1907 - val_accuracy: 0.9227 - lr: 0.0500
Epoch 4/20
1021/1021 [==============================] - 34s 34ms/step - loss: 0.1671 - accuracy: 0.9343 - val_loss: 0.2006 - val_accuracy: 0.9188 - lr: 0.0500
Epoch 5/20
1021/1021 [==============================] - 36s 35ms/step - loss: 0.1532 - accuracy: 0.9400 - val_loss: 0.2379 - val_accuracy: 0.9100 - lr: 0.0500
Epoch 6/20
1021/1021 [==============================] - 34s 33ms/step - loss: 0.1432 - accuracy: 0.9444 - val_loss: 0.1714 - val_accuracy: 0.9303 - lr: 0.0500
Epoch 7/20
1021/1021 [========================

In [21]:
def detectSuicide(text):
    cleaned_text, length = clean_text(text)
    tokens = tokenizer.texts_to_sequences(cleaned_text)
    paded = pad_sequences(tokens,maxlen=50)
    if suicide_model.predict([paded]) >=0.50:
#         print("contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 0
    else:
#         print("Doesn't contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 1
detectSuicide(["I don't want to live anymore"])

100%|██████████| 1/1 [00:00<00:00, 1005.11it/s]

1/1 [==============================] - 0s 38ms/step


0

In [25]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 6.8/6.8 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 199.8/199.8 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 2.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [23]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Transformer(nn.Module):
    def __init__(self, num_classes):
        super(Transformer, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output[1]
        logits = self.fc(pooled_output)
        return logits

In [24]:
model = Transformer(num_classes=7)
model.load_state_dict(torch.load("model.pth", map_location= torch.device("cpu")))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [25]:
text = "The only reason I haven’t killed myself yet is because of my mom and brothers. If I didn’t have them I’d be dead long ago. "
tokens = bert_tokenizer.encode_plus(
    text,
    max_length=512,
    
    add_special_tokens=True,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
)
output = model((tokens['input_ids']), (tokens['attention_mask']))
predicted_label = torch.argmax(output, dim=1).item()
print(predicted_label)

0


In [26]:
detectSuicide(["i do not want to live anymore"])

100%|██████████| 1/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 29ms/step


0

In [27]:
i need help just help me im crying so hard. There is no point living now
I am miserable .My life is over now.
The only reason I haven’t killed myself yet is because of my mom and brothers.If I didn’t have them I’d be dead

SyntaxError: invalid syntax (2067087818.py, line 1)

In [28]:
suicide_model.save("model_2.h5")

In [29]:
data["text"][20]

'Put opinions here Just say any controversial opinions you have that you want to share. And don’t make fun of others’ opinions. If you don’t agree just ignore.'

In [30]:
list= ["i need helpjust help me im crying so hard", "Oh Fuck! can't take this anymore.I am tired of this.", "This is it , I can't live anymore.", "I am gonna hurt myself",
      "I do not want to self harm again", "It ends tonight.I can’t do it anymore.I quit.", "I am endng it .This ends now.",
      "what would be a painless method of suicide","I want to endmy life", "I want to commit suicide"]

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run()

In [42]:
pip install pymongo 

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [46]:
from flask import Flask, request, jsonify
app = Flask(__name__)
from pymongo import MongoClient
import json

client = MongoClient('mongodb://localhost:27017')
db = client.serene
collection = db.innovates

@app.route('/reg', methods=['POST'])
def receive_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()  # Get the JSON data sent from the client
    # Process the data
    #

    result = collection.insert_one(data)
    res= {"data" : "hello"}
    return jsonify(res)

@app.route('/model', methods=['POST'])
def model_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id = data['id']
    id = str(id)
    
    mail = data['mail']
    #print(id)
    print(mail)
    print(id)
    m=detectSuicide([id])   
    print(m)
    if m==1:
        text = id
        print(text)
        tokens = bert_tokenizer.encode_plus(
                        text,
                        max_length=512,
    
                        add_special_tokens=True,
                        return_token_type_ids=False,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                    )
        output = model((tokens['input_ids']), (tokens['attention_mask']))
        m = int(torch.argmax(output, dim=1).item())
        m=m+1
    
    print(m)
    print(mail)
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB

#     result = collection.aggregate([
#     {"$unwind": "$Mood"},
#     {"$group": {"Email": mail, "average": {"$avg": "Mood"}}}
#     ])
    
#     #for r in result:
#     collection.update_one({"Email": mail}, {"$set": {"fitness": result}})

    lst = collection.find_one({'Email': mail })
#     mylist = lst['Mood']

    
    if lst is not None:
        # extract mylist array from document
        mylist = lst['Mood']
        updated_doc = collection.find_one_and_update(
        {'Email': mail},
        {'$push': {'Mood': m}}
        )
        
        total=0
        for i in mylist:
            total+=i
        avg=total/i+1

        #filter = {'Email': ObjectId('document_id')}
        update = {'$set': {'fitness': 'avg'}}
        result = collection.update_one(lst, update)
            
        json_data = jsonify(m)
        return json_data

    # res = {"data": "hello"}
    # return jsonify(id)

    else:
        # return error response if document not found
        return jsonify({'error': 'Document not found'})

    # Send a JSON response back to the client
# Press the green button in the gutter to run the script.

@app.route('/login', methods=['POST'])
def login_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id1 = data['id']
    pas = data['pass']
    
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB.
    lst = collection.find_one({'Email': id1})
    print(id1)
    if lst is not None:
        # extract mylist array from document
        checkpass = lst['Password']
        if(str(pas)==checkpass):
            return jsonify(id1)
            print("hello")
        
        else:
            code={'error' : 'existing user'}
            return jsonify(code)
    else:
        cod = {'error' : 'User doest exists'}
        return  jsonify((cod))
        # json_data = jsonify(mylist)
        # return json_data
@app.route('/')
def hello_world():
    return 'Hello, World!'


if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2023 00:02:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 00:02:41] "POST /login HTTP/1.1" 200 -


123@gmail.com


127.0.0.1 - - [06/Apr/2023 00:02:41] "GET /favicon.ico HTTP/1.1" 404 -


undefined
i am happy


...

100%|██████████| 1/1 [00:00<00:00, 142.95it/s]

1/1 [==============================] - 0s 46ms/step



127.0.0.1 - - [06/Apr/2023 00:04:27] "POST /model HTTP/1.1" 200 -


0
0
[object Object]


127.0.0.1 - - [06/Apr/2023 00:06:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 00:07:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 00:07:26] "POST /reg HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
app = Flask(__name__)
from pymongo import MongoClient
import json

client = MongoClient('mongodb://localhost:27017')
db = client.serene
collection = db.users

@app.route('/reg', methods=['POST'])
def receive_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()  # Get the JSON data sent from the client
    # Process the data
    #

    result = collection.insert_one(data)
    res= {"data" : "hello"}
    return jsonify(res)
@app.route('/model', methods=['POST'])
def model_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id = data['id']
    id = str(id)
    mail = data['mail']
    print(id)
    
    df = pd.read_csv('pulse.csv')
    dff=df['rate']

    m = detectSuicide([id])   
    for i in dff:
        up_doc = collection.find_one_and_update(
            {'Email': mail},
            {'$push': {'Pulse': i}}
        )
        
    print(m)
    if m==1:
        text = id
        print(text)
        tokens = bert_tokenizer.encode_plus(
                        text,
                        max_length=512,
    
                        add_special_tokens=True,
                        return_token_type_ids=False,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                    )
        output = model((tokens['input_ids']), (tokens['attention_mask']))
        m = int(torch.argmax(output, dim=1).item())
        m=m+1
    
    print(m)
    print(mail)
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB
    updated_doc = collection.find_one_and_update(
        {'Email': mail},
        {'$push': {'Mood': m}}
    )

    lst = collection.find_one({'Email': mail })
    if lst is not None:
        # extract mylist array from document
#         mylist = lst['Mood']
        json_data = jsonify(m)
        return json_data

    # res = {"data": "hello"}
    # return jsonify(id)

    else:
        # return error response if document not found
        return jsonify({'error': 'Document not found'})

    # Send a JSON response back to the client
# Press the green button in the gutter to run the script.

@app.route('/login', methods=['POST'])
def login_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id = data['id']
    pas = data['pass']
    
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB.
    lst = collection.find_one({'Email': id})
    if lst is not None:
        # extract mylist array from document
        checkpass = lst['Password']
        if(str(pas)==checkpass):
            return jsonify(id)
        else:
            code={'error' : 'existing user'}
            return jsonify(code)
    else:
        cod = {'error' : 'User doest exists'}
        return  jsonify((cod))
        # json_data = jsonify(mylist)
        # return json_data

@app.route('/graph', methods=['POST'])
def chart_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id1 = data
    
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB.
    lst = collection.find_one({'Email': id1})
    if lst is not None:
        # extract mylist array from document
        md = lst['Mood']
        return jsonify(md)
    else:
        cod = {'error' : 'User doest exists'}
        return  jsonify((cod))        
        
        
@app.route('/')
def hello_world():
    return 'Hello, World!'


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2023 07:04:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:04:11] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:04:11] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:04:11] "GET /home.css HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:04:11] "GET /playground_assets/pastedimage-no9b-1500h.png HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:08:20] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:09:23] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:09:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:09:28] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:09:28] "POST /graph HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 07:09:28] "GET /home.css HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:09:28] "GET /playground_assets/pastedimage-no9b-1500h.png HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 07:10:30] "POST /graph HTTP/1.1